In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from PIL import Image
import scipy.ndimage
import os 

Loading the sample image and dispalying it in color

In [ ]:
color_img = np.asarray(Image.open("test_image_2.jpg"))
plt.figure(figsize = (5,5))
plt.title("Sample Image")
plt.imshow(color_img)

Converting the Image to Grayscale 

In [ ]:
def grayscale(img):
    r,g,b, = img[:,:,0], img[:,:,1], img[:,:,2]
    gray_img = 0.299*r+0.587*g+0.114*b
    return gray_img

gray_img = grayscale(color_img)
plt.figure(figsize = (5,5))
plt.title("Grayscale Image")
plt.imshow(gray_img, cmap= "gray")

Applying Gaussian Filter to smooth Noise from img

In [ ]:
def gaussian_filter(filter_size, sigma):
    kernel = np.zeros((filter_size,filter_size))
    norm_factor = 1/(2.0*np.pi*sigma**2)
    k = (filter_size-1)/2.0

    for i in range(filter_size):
        for j in range(filter_size):
            x = (i-(k+1))**2
            y = (j-(k+1))**2
            kernel[i,j] = norm_factor*np.exp(-(x+y)/(2*sigma**2))
            
    return kernel


g_filter = gaussian_filter(5,1.4) ##using a 5x5 filter and a lambda value of 1
gaussian_blur_img = scipy.ndimage.convolve(gray_img,g_filter)

plt.figure(figsize=(5,5))
plt.title("Gaussian Blur Applied")
plt.imshow(gaussian_blur_img,cmap="gray")


To detect edges we need to detect contrast in pixels, there are multiple different edge detection operators that we can convolve the img with to detect contrast and hence edges. I'm going to be trying Sobel, Roberts, and Prewitts, operators.

In [ ]:
def edge_detect_operation(img, operator):
    #horizontal edges
    if operator == "sobel":
        Mx = np.array([[-1,0,1],
                    [-2,0,2],
                    [-1,0,1]])
        #vertical edges
        My = np.array([[-1,-2,-1],
                    [0,0,0],
                    [1,2,1]])
    elif operator == "prewitt":
        Mx = np.array([[-1,0,1],
                    [-1,0,1],
                    [-1,0,1]])
        #vertical edges
        My = np.array([[-1,-1,-1],
                    [0,0,0],
                    [1,1,1]])
    elif operator == "robert":
        Mx = np.array([[1,0],
                    [0,-1]])
        #vertical edges
        My = np.array([[0,1],
                    [-1,0]])
    
        
    Gx = scipy.ndimage.convolve(img,Mx)
    Gy = scipy.ndimage.convolve(img,My)

    G = np.hypot(Gx,Gy)

    theta = np.arctan2(Gy,Gx)

    return G, theta

sobel_img , sobel_theta= edge_detect_operation(gray_img,"sobel")
prewitt_img , prewitt_theta= edge_detect_operation(gray_img,"prewitt")
robert_img, robert_theta = edge_detect_operation(gray_img,"robert")

_, imgp = plt.subplots(1,3, figsize = (15,5))

imgp[0].set_title("Sobel Operator Image")
imgp[0].imshow(sobel_img,cmap = "gray")
imgp[0].axis("off")


imgp[1].set_title("Prewitt Operator Image")
imgp[1].imshow(prewitt_img, cmap = "gray")
imgp[1].axis("off")

imgp[2].set_title("Robert operator Image")
imgp[2].imshow(robert_img, cmap = "gray")
imgp[2].axis("off")

Next we will apply non-max supression in order to thin out the edges. From the previous calculations we have the gradients for each of the pixel values, we can use the gradient data to decide whether to keep or remove a pixel. For the rest of the caluclation im going to the sobel filiter image.

In [ ]:
#post sobel filter image and the gradients for each of the pixels
# directions is in degrees
def non_max_supression(img, angles):
    res = np.zeros_like(img)
    M, N = img.shape  #matrix of shape MxN

    N1 = 0 
    N2 = 0 

    for i in range(M-1):
        for j in range(N-1):
            if angles[i,j]>= -22.5 and angles[i,j]<=22.5 or angles[i,j]>= 157.5 and angles[i,j]<= -157.5:
                N1 = img[i,j+1]
                N2 = img[i,j-1]
            elif angles[i,j]>= 22.5 and angles[i,j]<=67.5 or angles[i,j]>= -157.5 and angles[i,j]<= -112.5:
                N1 = img[i-1,j+1]
                N2 = img[i+1,j-1]
            elif angles[i,j]>= 67.5 and angles[i,j]<=112.5 or angles[i,j]>= -112.5 and angles[i,j]<= -67.5:
                N1 = img[i+1,j]
                N2 = img[i-1,j]
            elif angles[i,j]>= 112.5 and angles[i,j]<=157.5 or angles[i,j]>= -67.5 and angles[i,j]<= -22.5:
                N1 = img[i+1,j+1]
                N2 = img[i-1,j-1]
    
            if img[i,j]> N1 and img[i,j]>N2:
                res[i,j] = img[i,j]
    return res

non_max_suppressed_img = non_max_supression(sobel_img, np.degrees(sobel_theta))


plt.imshow(non_max_suppressed_img,cmap= "gray")
plt.title("Image After Non-Max-Supression")
plt.axis("off")
plt.figure(figsize = (15,5))

Double Threshold

In [ ]:
def double_threshold(img,high):
    res = np.zeros_like(img)
    high_threshold = 255 * high 

    #low will be 1/3 of high
    low_threshold =  high_threshold*1/3

    strong_x,strong_y = np.where(img>high_threshold) 
    weak_x,weak_y = np.where((img>low_threshold) & (img<high_threshold))
    supressed_x, supressed_y = np.where(img<low_threshold)

    res[strong_x,strong_y] = 255
    res[weak_x,weak_y] = 20
    res[supressed_x,supressed_y] = 0 


    weak = (weak_x,weak_y)
    
    return res, weak

double_threshold_img, weak_pixels= double_threshold(non_max_suppressed_img,high = .3)

plt.imshow(double_threshold_img,cmap= "gray")
plt.title("Image After Double Threshold")
plt.axis("off")
plt.figure(figsize = (15,5))



# def double_threshold(img, low, high):


Hysteresis

In [ ]:
def hysteresis(img,weak):
    res = np.copy(img)
    M,N = np.shape(img)

    for point in zip(*weak):
        x,y = point
        try:
            if (res[x-1,y-1] == 255) or (res[x,y-1] == 255) or (res[x+1,y-1] == 255) or (res[x-1,y] == 255) or (res[x+1,y] == 255) or (res[x-1,y+1] == 255) or (res[x,y+1] == 255) or (res[x+1,y+1] == 255):
                res[x,y] = 255
        except IndexError as e:
            print(f"!!!Error at edge pixle {x,y}")

    return res


final_img = hysteresis(double_threshold_img,weak_pixels)

plt.imshow(final_img,cmap= "gray")
plt.title("Image after Hysteresis")
plt.axis("off")
plt.figure(figsize = (15,5))